## VPU based inferencing and deployment on IoT Edge device using Azure Machine Learning 

We will do deployment similar to the original: https://github.com/Azure-Samples/onnxruntime-iot-edge/tree/master/AzureML-OpenVINO



![End-to-end pipeline with ONNX Runtime](https://github.com/manashgoswami/byoc/raw/master/ONNXRuntime-AML.png)

In [ ]:
!python -m pip install --upgrade pip

In [ ]:
!pip install azureml-core azureml-contrib-iot azure-mgmt-containerregistry azure-cli
!az extension add --name azure-cli-iot-ext

In [ ]:
import os
print(os.__file__)

In [ ]:
# Check core SDK version number
import azureml.core as azcore

print("SDK version:", azcore.VERSION)

## 1. Setup the Azure Machine Learning Environment

### 1a AML Workspace : using existing config

In [ ]:
#Initialize Workspace 
from azureml.core import Workspace

ws = Workspace.from_config()

### 1.2 AML Workspace : create a new workspace

Alternatively, you could create a workspace using `azureml.core`:

```
#Initialize Workspace 
from azureml.core import Workspace

### Change this cell from markdown to code and run this if you need to create a workspace 
### Update the values for your workspace below
ws=Workspace.create(subscription_id="<subscription-id goes here>",
                resource_group="<resource group goes here>",
                name="<name of the AML workspace>",
                location="<location>")
                
ws.write_config()
```

### 1.3 AML Workspace : initialize an existing workspace
Download the `config.json` file for your AML Workspace from the Azure portal

In [ ]:
#Initialize Workspace 
from azureml.core import Workspace

## existing AML Workspace in config.json
ws = Workspace.from_config('config.json')

In [ ]:
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

## 2. Setup the trained model to use in this example

### 2.1 Register the trained model in workspace from the ONNX Model Zoo

In [ ]:
import urllib.request
onnx_model_url = "https://onnxzoo.blob.core.windows.net/models/opset_8/tiny_yolov2/tiny_yolov2.tar.gz"
urllib.request.urlretrieve(onnx_model_url, filename="tiny_yolov2.tar.gz")
!tar xvzf tiny_yolov2.tar.gz

In [ ]:
from azureml.core.model import Model

model = Model.register(workspace = ws, 
                       model_path = "./tiny_yolov2/Model.onnx",
                       model_name = "Model.onnx",
                       tags = {"data": "Imagenet", "model": "object_detection", "type": "TinyYolo"},
                       description = "real-time object detection model from ONNX model zoo")

### 2.2 Load the model from your workspace model registry
For e.g. this could be the ONNX model exported from your training experiment

In [ ]:
from azureml.core.model import Model
model = Model(name='Model.onnx', workspace=ws)

## 3. Create the application container image
This container is the IoT Edge module that will be deployed on the UP<sup>2</sup> device. 
    1. This container is using a pre-build base image for ONNX Runtime.
    2. Includes a `score.py` script, Must include a `run()` and `init()` function. The `init()` is entrypoint that reads the camera frames from /device/video0. The `run()` function is a dummy module to satisfy AML-sdk checks.
    3. `amlpackage_inference.py` script which is used to process the input frame and run the inference session and
    4. the ONNX model, label file used by the ONNX Runtime

In [ ]:
%%writefile score.py
# Copyright (c) Microsoft. All rights reserved.
# Licensed under the MIT license. See LICENSE file in the project root for
# full license information.


import sys
import time
import io
import csv


# Imports for inferencing
import onnxruntime as rt
from amlpackage_inference import run_onnx
import numpy as np
import cv2

# Imports for communication w/IOT Hub
from iothub_client import IoTHubModuleClient, IoTHubClientError, IoTHubTransportProvider
from iothub_client import IoTHubMessage, IoTHubMessageDispositionResult, IoTHubError
from azureml.core.model import Model

# Imports for the http server
from flask import Flask, request
import json

# Imports for storage
import os
# from azure.storage.blob import BlockBlobService, PublicAccess, AppendBlobService
import random
import string
import csv
from datetime import datetime
from pytz import timezone  
import time
import json

class HubManager(object):
    def __init__(
            self,
            protocol=IoTHubTransportProvider.MQTT):
        self.client_protocol = protocol
        self.client = IoTHubModuleClient()
        self.client.create_from_environment(protocol)

        # set the time until a message times out
        self.client.set_option("messageTimeout", MESSAGE_TIMEOUT)

    # Forwards the message received onto the next stage in the process.
    def forward_event_to_output(self, outputQueueName, event, send_context):
        self.client.send_event_async(
            outputQueueName, event, send_confirmation_callback, send_context)



def send_confirmation_callback(message, result, user_context):
    """
    Callback received when the message that we're forwarding is processed.
    """
    print("Confirmation[%d] received for message with result = %s" % (user_context, result))


def get_tinyyolo_frame_from_encode(msg):
    """
    Formats jpeg encoded msg to frame that can be processed by tiny_yolov2
    """
    #inp = np.array(msg).reshape((len(msg),1))
    #frame = cv2.imdecode(inp.astype(np.uint8), 1)
    frame = cv2.cvtColor(msg, cv2.COLOR_BGR2RGB)
    
    # resize and pad to keep input frame aspect ratio
    h, w = frame.shape[:2]
    tw = 416 if w > h else int(np.round(416.0 * w / h))
    th = 416 if h > w else int(np.round(416.0 * h / w))
    frame = cv2.resize(frame, (tw, th))
    pad_value=114
    top = int(max(0, np.round((416.0 - th) / 2)))
    left = int(max(0, np.round((416.0 - tw) / 2)))
    bottom = 416 - top - th
    right = 416 - left - tw
    frame = cv2.copyMakeBorder(frame, top, bottom, left, right,
                               cv2.BORDER_CONSTANT, value=[pad_value, pad_value, pad_value])
    
    frame = np.ascontiguousarray(np.array(frame, dtype=np.float32).transpose(2, 0, 1)) # HWC -> CHW
    frame = np.expand_dims(frame, axis=0)
    return frame

def run(msg):
    # this is a dummy function required to satisfy AML-SDK requirements.
    return msg

def init():
    # Choose HTTP, AMQP or MQTT as transport protocol.  Currently only MQTT is supported.
    PROTOCOL = IoTHubTransportProvider.MQTT
    DEVICE = 0 # when device is /dev/video0
    LABEL_FILE = "labels.txt"
    MODEL_FILE = "Model.onnx"
    global MESSAGE_TIMEOUT # setting for IoT Hub Manager
    MESSAGE_TIMEOUT = 1000
    LOCAL_DISPLAY = "OFF" # flag for local display on/off, default OFF

    
    # Create the IoT Hub Manager to send message to IoT Hub
    print("trying to make IOT Hub manager")
    
    hub_manager = HubManager(PROTOCOL)

    if not hub_manager:
        print("Took too long to make hub_manager, exiting program.")
        print("Try restarting IotEdge or this module.")
        sys.exit(1)

    # Get Labels from labels file 
    labels_file = open(LABEL_FILE)
    labels_string = labels_file.read()
    labels = labels_string.split(",")
    labels_file.close()
    label_lookup = {}
    for i, val in enumerate(labels):
        label_lookup[val] = i

    # get model path from within the container image
    model_path=Model.get_model_path(MODEL_FILE)
    
    # Loading ONNX model

    print("loading model to ONNX Runtime...")
    start_time = time.time()
    ort_session = rt.InferenceSession(model_path)
    print("loaded after", time.time()-start_time,"s")

    # start reading frames from video endpoint
    
    cap = cv2.VideoCapture(DEVICE)

    while cap.isOpened():
        _, _ = cap.read()
        ret, img_frame = cap.read()       
        if not ret:
            print('no video RESETTING FRAMES TO 0 TO RUN IN LOOP')
            cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
            continue
        
        """ 
        Handles incoming inference calls for each fames. Gets frame from request and calls inferencing function on frame.
        Sends result to IOT Hub.
        """
        try:
                        
            draw_frame = img_frame
            start_time = time.time()
            # pre-process the frame to flatten, scale for tiny-yolo
            
            frame = get_tinyyolo_frame_from_encode(img_frame)
            
            # run the inference session for the given input frame
            objects = run_onnx(frame, ort_session, draw_frame, labels, LOCAL_DISPLAY)
            
            # LOOK AT OBJECTS AND CHECK PREVIOUS STATUS TO APPEND
            num_objects = len(objects) 
            print("NUMBER OBJECTS DETECTED:", num_objects)                               
            print("PROCESSED IN:",time.time()-start_time,"s")            
            if num_objects > 0:
                output_IOT = IoTHubMessage(json.dumps(objects))
                hub_manager.forward_event_to_output("inferenceoutput", output_IOT, 0)
            continue
        except Exception as e:
            print('EXCEPTION:', str(e))
            continue

### 3.1 Include the dependent packages required by the application scripts

In [ ]:
from azureml.core.conda_dependencies import CondaDependencies

myenv = CondaDependencies()
myenv.add_pip_package("azure-iothub-device-client")
myenv.add_pip_package("numpy")
myenv.add_pip_package("opencv-python")
myenv.add_pip_package("requests")
myenv.add_pip_package("pytz")
myenv.add_pip_package("onnx")

with open("myenv.yml", "w") as f:
    f.write(myenv.serialize_to_string())

### 3.2 Build the custom container image with the ONNX Runtime + OpenVINO base image
This step uses pre-built container images with ONNX Runtime and the different HW execution providers. A complete list of base images are located [here](https://github.com/microsoft/onnxruntime/tree/master/dockerfiles#docker-containers-for-onnx-runtime).

In [ ]:
from azureml.core.image import ContainerImage
from azureml.core.model import Model

# Set the web service configuration (using default here)
from azureml.core.model import InferenceConfig
#from azureml.core.webservice import AksWebservice
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.environment import Environment, DEFAULT_GPU_IMAGE

useContainerImage = True

if useContainerImage:
    openvino_image_config = ContainerImage.image_configuration(execution_script = "score.py",
                                                           runtime = "python",
                                                           dependencies=["labels.txt", "amlpackage_inference.py"],
                                                           conda_file = "myenv.yml",
                                                           description = "TinyYolo ONNX Runtime inference container",
                                                           tags = {"demo": "onnx"})

    # Use the ONNX Runtime + OpenVINO base image for Intel MovidiusTM USB sticks
    openvino_image_config.base_image = "mcr.microsoft.com/azureml/onnxruntime:latest-openvino-myriad" 

    # For the Intel Movidius VAD-M PCIe card use this:
    # openvino_image_config.base_image = "mcr.microsoft.com/azureml/onnxruntime:latest-openvino-vadm"

    openvino_image = ContainerImage.create(name = "name-of-image",
                              # this is the model object
                              models = [model],
                              image_config = openvino_image_config,
                              workspace = ws)

    # Alternative: Re-use an image that you have already built from the workspace image registry
    # openvino_image = ContainerImage(name = "<name-of-image>", workspace = ws)

else:
    env = Environment('deploytoedgeenv')
    # Please see [Azure ML Containers repository](https://github.com/Azure/AzureML-Containers#featured-tags)
    # for open-sourced GPU base images.
    
    env.docker.base_image = "mcr.microsoft.com/azureml/onnxruntime:latest-openvino-myriad"
    #env.docker.base_image = DEFAULT_GPU_IMAGE

    env.python.conda_dependencies = CondaDependencies.create(
                            conda_packages=['tensorflow-gpu==1.12.0','numpy'],
                            pip_packages=['azureml-defaults','azure-iothub-device-client','numpy','opencv-python','requests','pytz','onnx']
                            )

    inference_config = InferenceConfig(entry_script="score.py", environment=env)
    imagename= "myopenvino-myriad"
    #imagename= "myopenvino"
    imagelabel="1.0"


In [ ]:
if useContainerImage:
    openvino_image.wait_for_creation(show_output = True)
    if openvino_image.creation_state == 'Failed':
        print("Image build log at: " + openvino_image.image_build_log_uri)
else:
    package = Model.package(ws, [model], inference_config=inference_config,image_name=imagename, image_label=imagelabel)
    package.wait_for_creation(show_output=True)

In [ ]:
if useContainerImage:
    if openvino_image.creation_state != 'Failed':
        print("Image URI at: " +openvino_image.image_location)
else:
    print("ACR:", package.get_container_registry)
    print("Image:", package.location)

## 4. Deploy to the UP<sup>2</sup> device using Azure IoT Edge

### 4.1 Login with the Azure subscription to provision the IoT Hub and the IoT Edge device

In [ ]:
!az login 
!az account set --subscription $ws.subscription_id 


In [ ]:
# confirm the account
!az account show

### 4.2 Specify the IoT Edge device details

In [ ]:
# Parameter list to configure the IoT Hub and the IoT Edge device

# Pick a name for what you want to call the module you deploy to the camera
module_name = "module-name-here"

# Resource group in Azure 
resource_group_name= ws.resource_group
iot_rg=resource_group_name

# Azure region where your services will be provisioned
iot_location="location-here"

# Azure IoT Hub name
iot_hub_name="name-of-IoT-Hub"

# Pick a name for your camera
iot_device_id="name-of-IoT-Edge-device"

# Pick a name for the deployment configuration
iot_deployment_id="Infernce Module from AML"

### 4.2a Optional: Provision the IoT Hub, create the IoT Edge device and Setup the Intel UP<sup>2</sup> AI Vision Developer Kit

In [ ]:
!az iot hub create --resource-group $resource_group_name --name $iot_hub_name --sku S1

In [ ]:
# Register an IoT Edge device (create a new entry in the Iot Hub)
!az iot hub device-identity create --hub-name $iot_hub_name --device-id $iot_device_id --edge-enabled

In [ ]:
!az iot hub device-identity show-connection-string --hub-name $iot_hub_name --device-id $iot_device_id 

The following steps need to be executed in the device terminal

1. Open the IoT edge configuration file in UP<sup>2</sup> device to update the IoT Edge device *connection string*
    
    `sudo nano /etc/iotedge/config.yaml`
    
        provisioning:
        source: "manual"
        device_connection_string: "<ADD DEVICE CONNECTION STRING HERE>"

2. To update the DPS TPM provisioning configuration:

        provisioning:
        source: "dps"
        global_endpoint: "https://global.azure-devices-provisioning.net"
        scope_id: "{scope_id}"
        attestation:
        method: "tpm"
        registration_id: "{registration_id}"

3. Save and close the file. `CTRL + X, Y, Enter

    
4. After entering the privisioning information in the configuration file, restart the *iotedge* daemon
    
    `sudo systemctl restart iotedge`
    
    
5. We will show the object detection results from the camera connected (`/dev/video0`) to the UP<sup>2</sup> on the display. Update your .profile file:
    
    `nano ~/.profile`
    
   add the following line to the end of file

    __xhost +__

### 4.3 Construct the deployment file

In [ ]:
# create the registry uri
container_reg = ws.get_details()["containerRegistry"]
reg_name=container_reg.split("/")[-1]
container_url = "\"" + openvino_image.image_location + "\","
subscription_id = ws.subscription_id
print('{}'.format(openvino_image.image_location), "<-- this is the URI configured in the IoT Hub for the device")
print('{}'.format(reg_name))
print('{}'.format(subscription_id))

In [ ]:
from azure.mgmt.containerregistry import ContainerRegistryManagementClient
from azure.mgmt import containerregistry
client = ContainerRegistryManagementClient(ws._auth,subscription_id)
result= client.registries.list_credentials(resource_group_name, reg_name, custom_headers=None, raw=False)
username = result.username
password = result.passwords[0].value

#### Create the `deplpyment.json` with the AML image registry details
We have provided here a sample deployment template this reference implementation.

In [ ]:
file = open('./aml-deployment.template.json')
contents = file.read()
contents = contents.replace('__AML_MODULE_NAME', module_name)
contents = contents.replace('__AML_REGISTRY_NAME', reg_name)
contents = contents.replace('__AML_REGISTRY_USER_NAME', username)
contents = contents.replace('__AML_REGISTRY_PASSWORD', password)
contents = contents.replace('__AML_REGISTRY_IMAGE_LOCATION', openvino_image.image_location)
with open('./deployment.json', 'wt', encoding='utf-8') as output_file:
    output_file.write(contents)

### 4.4 Push the *deployment* to the IoT Edge device

In [ ]:
print("Pushing deployment to IoT Edge device")

In [ ]:
print ("Set the deployement") 
!az iot edge set-modules --device-id $iot_device_id --hub-name $iot_hub_name --content deployment.json

### 4.5 Monitor IoT Hub Messages

In [ ]:
!az iot hub monitor-events --hub-name $iot_hub_name -y

## 5. CLEANUP

In [ ]:
!rm score.py deployment.json myenv.yml